In [1]:
import pickle



#数据集用的是phone1.json
def bert_QA(file, question):
    import torch
    import json
    from transformers import BertForQuestionAnswering
    from transformers import BertTokenizer
    
    model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    
    json_file = file

    # 将JSON文件中的"output"字段转换为字符串
    with open(json_file, 'r') as f:
        data = json.load(f)
        output_str = ''
        for obj in data:
            output_str += obj['output'] + '\n'
    answer_text = output_str
    
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    #Here We Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
      # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
            # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    print('Answer: "' + answer + '"')
 
question = "What are the best phones for gaming?"
bert_QA('phone1.json',question)

with open('bert_QA.pickle', 'wb') as f:
    pickle.dump(bert_QA, f)
    

D:\ANACONDA\envs\pytorch2\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Answer: "asus rog phone 5 , samsung galaxy s21 ultra , iphone 12 pro max , oneplus 9 pro , and xiaomi black shark 4"
